In [2]:
import pandas as pd

# Load Model

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=False)
resnet18.fc = torch.nn.Linear(in_features=512, out_features=3)
resnet18.load_state_dict(torch.load('reference_balanced.pth'))
resnet18.eval()

# Calculate Embedding for each COVID Image

In [ ]:
# https://becominghuman.ai/extract-a-feature-vector-for-any-image-with-pytorch-9717561d1d4c
import torchvision.transforms as transforms
from torch.autograd import Variable

resnet18.eval()
resnet18.to('cpu')

scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_embedding(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    embedding = {}
    # 4. Define a function that will copy the output of a layer
    def copy_data_to_dict(m, i, o):
        embedding['out'] = o.data
    # 5. Attach that function to our selected layer
    h = resnet18._modules.get('avgpool').register_forward_hook(copy_data_to_dict)
    # 6. Run the model on our transformed image
    resnet18(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return embedding['out']

x = get_embedding(covid_df.at[0, 'PATH'])


In [5]:
valencia_covid_df = pd.read_csv('../datasets/bimcv-covid19/bimcv-covid19.csv', index_col=0)
valencia_covid_df

,Subject,Session,Date,Age,Sex,Filename,Path,Is frontal
0,sub-S03533,ses-E07501,20200328,052Y,M,mod-rx/sub-S03533_ses-E07501_bp-chest_run-1_cr...,../datasets/bimcv-covid19/sub-S03533/ses-E0750...,y
1,sub-S03533,ses-E07501,20200328,052Y,M,mod-rx/sub-S03533_ses-E07501_bp-chest_run-2_cr...,../datasets/bimcv-covid19/sub-S03533/ses-E0750...,y
2,sub-S03909,ses-E08005,20200330,NaN,M,mod-rx/sub-S03909_ses-E08005_bp-chest_run-1_vp...,../datasets/bimcv-covid19/sub-S03909/ses-E0800...,y
3,sub-S03711,ses-E07526,20200322,NaN,F,mod-rx/sub-S03711_ses-E07526_bp-chest_run-1_ac...,../datasets/bimcv-covid19/sub-S03711/ses-E0752...,y
4,sub-S03711,ses-E07526,20200322,NaN,F,mod-rx/sub-S03711_ses-E07526_bp-chest_run-1_ac...,../datasets/bimcv-covid19/sub-S03711/ses-E0752...,n
...,...,...,...,...,...,...,...,...
3032,sub-S03288,ses-E07142,20200310,NaN,F,mod-rx/sub-S03288_ses-E07142_bp-chest_run-1_vp...,../datasets/bimcv-covid19/sub-S03288/ses-E0714...,y
3033,sub-S03288,ses-E06470,20200222,NaN,F,mod-rx/sub-S03288_ses-E06470_bp-chest_run-1_ac...,../datasets/bimcv-covid19/sub-S03288/ses-E0647...,y
3034,sub-S03288,ses-E06470,20200222,NaN,F,mod-rx/sub-S03288_ses-E06470_bp-chest_run-1_ac...,../datasets/bimcv-covid19/sub-S03288/ses-E0647...,n
3035,sub-S03492,ses-E07022,20200311,NaN,F,mod-rx/sub-S03492_ses-E07022_bp-chest_run-1_vp...,../datasets/bimcv-covid19/sub-S03492/ses-E0702...,y


In [ ]:
valencia_covid_df = pd.read_csv('../datasets/bimcv-covid19/bimcv-covid19.csv', index_col=0)

valencia_covid_df['embedding'] = valencia_covid_df.apply(lambda row: get_embedding(row['PATH']), axis=1)

## Calculate Similarity to an image

In [ ]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(pic_one_vector.unsqueeze(0),
              pic_two_vector.unsqueeze(0))
print('\nCosine similarity: {0}\n'.format(cos_sim))

valencia_covid_df['similarity'] = valencia_covid_df.apply(lambda row: cos(row['PATH'].unsqueeze(0),
              get_embedding(covid_df.at[0, 'PATH'])), axis=1)

In [ ]:
valencia_covid_df.sort_value(by='similarity')
valencia_covid_df.head()

## Display the Similar images

In [ ]:
im_frame = Image.open(valencia_covid_df.at[0, 'PATH'])
                                    
np_frame = np.array(im_frame)
plt.imshow(np_frame)

## Create map for subject to each scan

In [3]:
import os
from IPython.display import display, HTML

debug = False

root = '../datasets/bimcv-covid19/'
subject_dirs = [name for name in os.listdir(root) if '.' not in name and 'sub' == name[0:3]]
print('There are %d directories with patient data' % len(subject_dirs))
subject_to_sessions = {}
for subject_dir in subject_dirs:
    print(subject_dir)
    # find tsv file
    sessions_df = pd.read_csv('%s%s/%s_sessions.tsv' % (root, subject_dir, subject_dir), delimiter='\t')
    sessions_to_scans = {'sessions_df': sessions_df}
    # find png files per session
    if debug:
        display(HTML(sessions_df.to_html()))
    for session_id in sessions_df['session_id']:
        scans_df = pd.read_csv('%s%s/%s/%s_%s_scans.tsv' % (root, subject_dir, session_id, subject_dir, session_id), delimiter='\t')
        if debug:
            display(HTML(scans_df.to_html()))
        sessions_to_scans[session_id] = scans_df
        
    subject_to_sessions[subject_dir] = sessions_to_scans

There are 1273 directories with patient data
sub-S03533
sub-S03909
sub-S03711
sub-S04412
sub-S03947
sub-S03127
sub-S03193
sub-S03327
sub-S03908
sub-S04092
sub-S04269
sub-S03802
sub-S04151
sub-S03821
sub-S04281
sub-S03724
sub-S03169
sub-S03863
sub-S04212
sub-S04140
sub-S03848
sub-S04096
sub-S03733
sub-S04450
sub-S03851
sub-S03860
sub-S03619
sub-S03835
sub-S04154
sub-S03328
sub-S03519
sub-S03354
sub-S04392
sub-S03665
sub-S03202
sub-S04204
sub-S03523
sub-S03867
sub-S04439
sub-S03813
sub-S04095
sub-S03885
sub-S04172
sub-S04370
sub-S03267
sub-S04307
sub-S03266
sub-S04039
sub-S03493
sub-S03156
sub-S04117
sub-S03769
sub-S04501
sub-S04148
sub-S03743
sub-S03460
sub-S04174
sub-S03528
sub-S03196
sub-S03828
sub-S03102
sub-S04010
sub-S03708
sub-S04187
sub-S04167
sub-S03166
sub-S04003
sub-S04483
sub-S03181
sub-S03918
sub-S03956
sub-S03438
sub-S03255
sub-S03561
sub-S03512
sub-S03565
sub-S03938
sub-S03990
sub-S03997
sub-S03830
sub-S03854
sub-S04110
sub-S03119
sub-S03263
sub-S03530
sub-S03179
sub-S0384

sub-S03541
sub-S04449
sub-S03648
sub-S03829
sub-S03204
sub-S03750
sub-S04156
sub-S04221
sub-S03217
sub-S03788
sub-S03116
sub-S03616
sub-S03798
sub-S03546
sub-S03717
sub-S03149
sub-S03280
sub-S03902
sub-S04428
sub-S03939
sub-S04283
sub-S03924
sub-S04374
sub-S04484
sub-S03221
sub-S03646
sub-S03251
sub-S03627
sub-S03673
sub-S04303
sub-S03463
sub-S03186
sub-S04296
sub-S04132
sub-S03472
sub-S04060
sub-S04235
sub-S03995
sub-S04134
sub-S04308
sub-S03761
sub-S03614
sub-S03758
sub-S03429
sub-S03959
sub-S03764
sub-S04098
sub-S03864
sub-S04111
sub-S03800
sub-S04481
sub-S04026
sub-S04155
sub-S03422
sub-S03732
sub-S04343
sub-S03413
sub-S03414
sub-S03128
sub-S03899
sub-S03577
sub-S03477
sub-S03597
sub-S03101
sub-S04224
sub-S03223
sub-S03212
sub-S03481
sub-S04469
sub-S03485
sub-S03423
sub-S03133
sub-S03984
sub-S03191
sub-S03184
sub-S03866
sub-S03360
sub-S03203
sub-S03731
sub-S03333
sub-S03915
sub-S04255
sub-S03409
sub-S03737
sub-S04207
sub-S03625
sub-S03536
sub-S04040
sub-S04252
sub-S03595
sub-S03736

## Translate Annotation

In [ ]:
from google.cloud import translate


def translate_text(text="YOUR_TEXT_TO_TRANSLATE", project_id="cs230-295917"):
    """Translating Text."""

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "source_language_code": "en-US",
            "target_language_code": "fr",
        }
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        print("Translated text: {}".format(translation.translated_text))

In [11]:
sessions = subject_to_sessions[valencia_covid_df.at[0, 'Subject']].keys()
for medical_evaluation in subject_to_sessions[valencia_covid_df.at[0, 'Subject']]['sessions_df']['medical_evaluation']:
    print(translate_text(medical_evaluation))

ses-E07501
ses-E07183
